In [1]:
import numpy as np

In [2]:
%matplotlib inline
import matplotlib.pyplot as plt

In [3]:
from count import *

In [4]:
def score_diff_hist(data):
    hist = {}
    for (a, b), c in data.items():
        hist[a - b] = hist.get(a - b, 0) + c
    xx = np.arange(min(hist), max(hist) + 1)
    return xx, np.array([hist.get(x, 0) for x in xx])

In [10]:
def hoshitori(data_set, offset=0):
    hoshi_a = {}
    hoshi_b = {}
    for (name_a, name_b), data in data_set.items():
        if name_a not in hoshi_a:
            hoshi_a[name_a] = [0, 0, 0]
        if name_b not in hoshi_b:
            hoshi_b[name_b] = [0, 0, 0]
        for (sa, sb), c in data.items():
            score = sa - (sb + offset)
            if score == 0:
                hoshi_a[name_a][2] += c
                hoshi_b[name_b][2] += c
            elif score > 0:
                hoshi_a[name_a][0] += c
                hoshi_b[name_b][1] += c
            else:
                hoshi_a[name_a][1] += c
                hoshi_b[name_b][0] += c
    return hoshi_a, hoshi_b

def order_hoshi_by_win_rate(hoshi):
    for name in sorted(hoshi, key=lambda n: hoshi[n][0] / sum(hoshi[n]), reverse=True):
        h = hoshi[name]
        print(name, h, h[0] / sum(h))

In [141]:
import texttable

def to_table(data_set):
    names = list(set([name for name, _ in data_set]) | set([name for _, name in data_set]))
    names.sort()

    table = texttable.Texttable(max_width=500)
    table.set_cols_align(['l'] * (len(names) + 2))
    table.add_row([''] + names + [''])
    for name1 in names:
        row = [name1]
        total_win = total_lose = total_draw = 0
        for name2 in names:
            win = lose = draw = 0
            for (sa, sb), c in data_set.get((name1, name2), {}).items():
                if sa > sb:
                    win += c
                elif sa < sb:
                    lose += c
                else:
                    draw += c
            row.append(f'{win}-{lose}-{draw}\n({(100 * win / (win + lose + draw)):.1f})')
            total_win += win
            total_lose += lose
            total_draw += draw
        row.append(f'{total_win}-{total_lose}-{total_draw}\n({(100 * total_win / (total_win + total_lose + total_draw)):.1f})')
        table.add_row(row)
    
    row = ['']
    for name1 in names:
        win = lose = draw = 0
        for name2 in names:
            for (sa, sb), c in data_set[(name2, name1)].items():
                if sa < sb:
                    win += c
                elif sa > sb:
                    lose += c
                else:
                    draw += c
        row.append(f'{win}-{lose}-{draw}\n({(100 * win / (win + lose + draw)):.1f})')
    row.append('')
    table.add_row(row)
    return table.draw()

In [187]:
data = {}
for f in Path('.').glob('true/*'):
    count(f, data)

In [142]:
data = {}
count(Path('b.txt'), data)

In [188]:
hoshi_a, hoshi_b = hoshitori(data, offset=0)

In [189]:
order_hoshi_by_win_rate(hoshi_a)

dfs:pos:9 [600, 0, 0] 1.0
dfs:pos:5 [598, 1, 1] 0.9966666666666667
dfs:pos:7 [563, 35, 2] 0.9383333333333334
dfs:diff:7 [521, 73, 6] 0.8683333333333333
dfs:diff:9 [514, 76, 9] 0.8580968280467446
dfs:pos:3 [527, 92, 0] 0.851373182552504
mctree:14 [374, 87, 33] 0.757085020242915
mctree:12 [418, 130, 39] 0.7120954003407155
greedy [243, 324, 33] 0.405
random [29, 563, 8] 0.04833333333333333


In [190]:
order_hoshi_by_win_rate(hoshi_b)

dfs:pos:9 [256, 317, 17] 0.43389830508474575
dfs:pos:7 [249, 338, 9] 0.41778523489932884
mctree:14 [151, 427, 3] 0.25989672977624784
dfs:diff:9 [151, 389, 46] 0.257679180887372
dfs:pos:5 [143, 423, 30] 0.23993288590604026
mctree:12 [139, 434, 12] 0.2376068376068376
dfs:pos:3 [120, 477, 3] 0.2
dfs:diff:7 [83, 502, 3] 0.141156462585034
greedy [57, 529, 2] 0.09693877551020408
random [32, 551, 6] 0.05432937181663837


In [191]:
print(to_table(data))

+------------+------------+------------+-----------+------------+-----------+------------+----------+------------+-----------+----------+------------+
|            | dfs:diff:7 | dfs:diff:9 | dfs:pos:3 | dfs:pos:5  | dfs:pos:7 | dfs:pos:9  | greedy   | mctree:12  | mctree:14 | random   |            |
+------------+------------+------------+-----------+------------+-----------+------------+----------+------------+-----------+----------+------------+
| dfs:diff:7 | 57-3-0     | 60-0-0     | 60-0-0    | 45-9-6     | 60-0-0    | 0-60-0     | 60-0-0   | 59-1-0     | 60-0-0    | 60-0-0   | 521-73-6   |
|            | (95.0)     | (100.0)    | (100.0)   | (75.0)     | (100.0)   | (0.0)      | (100.0)  | (98.3)     | (100.0)   | (100.0)  | (86.8)     |
+------------+------------+------------+-----------+------------+-----------+------------+----------+------------+-----------+----------+------------+
| dfs:diff:9 | 60-0-0     | 56-4-0     | 56-4-0    | 45-15-0    | 41-14-5   | 32-24-4    | 60-